In [108]:
import torch as tr
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import random as rn

In [109]:
device = tr.device('cuda' if tr.cuda.is_available() else 'cpu')
# print(device)

In [110]:
#Force Field

X,Y = np.linspace(-0.75,0.75,43),np.linspace(-0.75,0.75,43)

# def U(x,y,U_0=0.4): #potential function, U_0 is the amplitude, x and y are the coordinates, Gradient needs to be calculated manually in forward pass.
#     if (x**2+y**2)**0.5<=0.5:
#         return tr.tensor((16*U_0*((x**2+y**2)**0.5-0.25)**2),dtype=tr.half)
#     else:
#         return tr.tensor(0,dtype=tr.half)

def U(x,y,U_0=0.4): #potential function, U_0 is the amplitude, x and y are the coordinates, Gradient needs to be calculated manually in forward pass.
    if (x**2+y**2)**0.5<=0.5:
        return 16*U_0*((x**2+y**2)**0.5-0.25)**2
    else:
        return 0
# Z = U(X,Y)
# F = tr.from_numpy(np.gradient(Z))
# print(F.shape)

In [111]:
#Toy Force Field
x = np.linspace(-0.75, 0.75, 100)
y = np.linspace(-0.75, 0.75, 100)
X, Y = np.meshgrid(x, y)

# Calculate the force field
force_field_x = -np.abs(Y)
force_field_y = np.zeros_like(X)

# Create the meshgrid for force field
F = tr.tensor(tr.from_numpy(np.stack((force_field_x, force_field_y), axis=2)),dtype=tr.float)


C:\Users\igonn\AppData\Local\Temp\ipykernel_17692\1769924650.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  F = tr.tensor(tr.from_numpy(np.stack((force_field_x, force_field_y), axis=2)),dtype=tr.float)


In [112]:
#Agent Class
class Agent(nn.Module):
    def __init__(self,x=tr.tensor(-0.5,dtype=tr.float),y=tr.tensor(0,dtype=tr.float),b=tr.tensor(0,dtype=tr.float)):#initialize agent at location (-0.5,-0.5)
        super(Agent, self).__init__()
        self.x=x #x coordinate
        self.y=y #y coordinate
        self.b=b #bias
        self.input = nn.Linear(5, 64,dtype=tr.float) #input layer
        self.hidden1 = nn.Linear(64, 64,dtype=tr.float) #hidden layer
        self.hidden2 = nn.Linear(64, 32, dtype = tr.float)
        self.output = nn.Linear(32, 4,dtype=tr.float) #hidden layer
        self.activation = nn.ReLU() #activation function

    def forward(self, x, network = None): #forward pass
        if network is None: network = self
        x1 = self.activation(self.input(x))
        x2 = self.activation(self.hidden1(x1))
        x3 = self.activation(self.hidden2(x2))
        x4 = self.activation(self.output(x3))
        return x4
    

    def move(self,Q,e):
        if rn.random()<e:
            L = rn.randint(0,3)
        else: L = tr.argmax(Q)
        
        dx,dy = 0,0
        if L==0: dx = 0.0357 #up
        elif L==1: dx = -0.0357 #down
        elif L==2: dy = 0.0357 #right
        elif L==3: dy = -0.0357 #left
        
        if not -0.75<=self.x+dx<=0.75:
            dx=0
        if not -0.75<=self.y+dy<=0.75:
            dy=0
        self.x+=dx
        self.y+=dy

def R(x,y,F):
    dt = 0.0357/(F+1e-8)
    return -0.5*dt/1000-0.5*tr.norm(tr.tensor([x-0.5,y]),p=2)

# def R(x,y):
#     return -tr.norm(tr.tensor([x-0.5,y]),p=2)

In [113]:
#Simulation
agent = Agent().to(device)
t_agent = Agent().to(device)
t_agent.load_state_dict(agent.state_dict())
t_agent.eval()
t_update = 100
optimizer = tr.optim.Adam(agent.parameters(), lr=0.001)
dT = 0
g = 0.8
max_steps = 2000
while dT<max_steps:
    dT+=1
    if dT%t_update==0:#update target agent every 100 steps
        t_agent.load_state_dict(agent.state_dict())
    
    Q1 = agent.forward(tr.tensor([agent.x,agent.y,-np.abs(agent.y),tr.tensor(0,dtype=tr.float),agent.b]).to(device)) #Q value

    e = 1-dT/max_steps #epsilon
    agent.move(Q1,e)      	    #move agent

    Q2 = agent.forward(tr.tensor([agent.x,agent.y,-tr.abs(agent.y),tr.tensor(0,dtype=tr.float),agent.b]).to(device),network=t_agent) #Q value
    tQ = R(agent.x,agent.y,agent.y)+ g*tr.max(Q2)         #target Q value
    criterion = nn.MSELoss().to(device)     #1/2*(tQ-Q1)**2 #loss
    loss = criterion(tQ,tr.max(Q1))         #loss
    
    optimizer.zero_grad()                   #zero gradients
    loss.backward()                         #backpropagate
    optimizer.step()                        #update weights
    if (tr.round(agent.x,decimals = 1),tr.round(agent.y,decimals = 1)) == (0.5000,0.5000):
        print('success')
        break
print(agent.x,agent.y)
# print(tr.round(agent.x,decimals = 1),tr.round(agent.y,decimals = 1))

tensor(0.7000) tensor(0.3000)
